# TensorFlow 模型建立与训练
https://tf.wiki/zh/basic/models.html

## 循环神经网络（RNN）
https://tf.wiki/zh/basic/models.html#rnn

循环神经网络（Recurrent Neural Network, RNN）是一种适宜于处理序列数据的神经网络，被广泛用于语言模型、文本生成、机器翻译等。

首先，还是实现一个简单的 DataLoader 类来读取文本，并以字符为单位进行编码。设字符种类数为 num_chars ，则每种字符赋予一个 0 到 num_chars - 1 之间的唯一整数编号 i。

In [1]:
import tensorflow as tf
import numpy as np

class DataLoader():
    def __init__(self):
        path = tf.keras.utils.get_file('nietzsche.txt',
            origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
        with open(path, encoding='utf-8') as f:
            self.raw_text = f.read().lower()
        self.chars = sorted(list(set(self.raw_text)))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))
        self.text = [self.char_indices[c] for c in self.raw_text]

    def get_batch(self, seq_length, batch_size):
        seq = []
        next_char = []
        for i in range(batch_size):
            index = np.random.randint(0, len(self.text) - seq_length)
            seq.append(self.text[index:index+seq_length])
            next_char.append(self.text[index+seq_length])
        return np.array(seq), np.array(next_char)       # [batch_size, seq_length], [num_batch]


In [2]:
class RNN(tf.keras.Model):
    def __init__(self, num_chars, batch_size, seq_length):
        super().__init__()
        self.num_chars = num_chars
        self.seq_length = seq_length
        self.batch_size = batch_size
        self.cell = tf.keras.layers.LSTMCell(units=256)
        self.dense = tf.keras.layers.Dense(units=self.num_chars)

    def call(self, inputs, from_logits=False):
        inputs = tf.one_hot(inputs, depth=self.num_chars)       # [batch_size, seq_length, num_chars]
        state = self.cell.get_initial_state(batch_size=self.batch_size, dtype=tf.float32)
        for t in range(self.seq_length):
            output, state = self.cell(inputs[:, t, :], state)
        logits = self.dense(output)
        if from_logits:
            return logits
        else:
            return tf.nn.softmax(logits)

    def predict(self, inputs, temperature=1.):
        batch_size, _ = tf.shape(inputs)
        logits = self(inputs, from_logits=True)
        prob = tf.nn.softmax(logits / temperature).numpy()
        return np.array([np.random.choice(self.num_chars, p=prob[i, :])
                         for i in range(batch_size.numpy())])

In [3]:
num_batches = 1000
seq_length = 40
batch_size = 50
learning_rate = 1e-3

In [4]:
data_loader = DataLoader()
model = RNN(num_chars=len(data_loader.chars), batch_size=batch_size, seq_length=seq_length)
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
for batch_index in range(num_batches):
    X, y = data_loader.get_batch(seq_length, batch_size)
    with tf.GradientTape() as tape:
        y_pred = model(X)
        loss = tf.keras.losses.sparse_categorical_crossentropy(y_true=y, y_pred=y_pred)
        loss = tf.reduce_mean(loss)
        if (batch_index + 1) % 100 == 0:
            print("batch %d: loss %f" % (batch_index, loss.numpy()))
    grads = tape.gradient(loss, model.variables)
    optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))

batch 99: loss 2.985571
batch 199: loss 2.846085
batch 299: loss 2.986976
batch 399: loss 2.905682
batch 499: loss 2.350620
batch 599: loss 2.478311
batch 699: loss 2.398890
batch 799: loss 2.610779
batch 899: loss 2.781663
batch 999: loss 2.536434


In [6]:
X_, _ = data_loader.get_batch(seq_length, 1)
for diversity in [0.2, 0.5, 1.0, 1.2]:
    X = X_
    print("diversity %f:" % diversity)
    for t in range(400):
        y_pred = model.predict(X, diversity)
        print(data_loader.indices_char[y_pred[0]], end='', flush=True)
        X = np.concatenate([X[:, 1:], np.expand_dims(y_pred, axis=1)], axis=-1)
    print("\n")

diversity 0.200000:
ere the the the the the he the the the the the the the the the the the the the and and of the the the the the ind of mathe the the the the the the the the the at ous of the and ous of of the the and of the the and of the the he the and of the the se the the andere the fere the the the the the the the the seese the the the the the the the the and ond on the the the the the the the the he the the th

diversity 0.500000:
e ere what
of arecere  on the the thas and or the the alithen the wion tithe
ferout oust of whe the the mouthe he the his so the there more lution the pite deriom the nouss ousas the here the aude tous outhers and
marte the the caeder of the ande- and ind of of the we thit outinty the siche
er ind ou te the ousiol of inpe aon mothoutipertere toa the ceateeres ot oo the aude the the pereos the worl

diversity 1.000000:
ese pedethaa
e bimman- atdony
s ans anof eaf andor oricly at oo uthethel
arate; cacfeoo ind mophemi
ongl tuthelwumoupeoud
andnly fopeld 